In [47]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import re

In [48]:
xls = pd.ExcelFile("Product Matching Dataset.xlsx")

In [49]:
master_df = pd.read_excel(xls, "Master File")
dataset_df = pd.read_excel(xls, "Dataset")

In [50]:
dataset_df.head()

,sku,marketplace_product_name_ar,seller_item_name,price
0,1322,استوهالت 40 مجم 14 كبسول,ESTOHALT 40 MG 14 CAP,56.5
1,1322,استوهالت 40 مجم 14 كبسول,استوهالت 40 مجم 14 ك,56.5
2,1322,استوهالت 40 مجم 14 كبسول,استوهالت 40 مجم 14 ك,56.5
3,1322,استوهالت 40 مجم 14 كبسول,استوهالت 40 مجم 14 ك,56.5
4,1322,استوهالت 40 مجم 14 كبسول,استوهالت 40 مجم 14 ك,56.5


In [51]:
master_df.head()

,sku,product_name,product_name_ar,price
0,279,ANAFRONIL 75 MG 20 TAB,انافرونيل 75 مجم اس ار 20 قرص,75.0
1,2282,LOPRECOUGH SYRUP 100 ML,لوبريكاف شراب 100 مل,28.5
2,4331,TOMEX PLUS 50 TAB,تومكس بلس 50 قرص,60.0
3,1022,TAROLIMUS 0.03% OINT. 15 GM,تاروليمس 0.03 % مرهم 15 جم,129.0
4,116,GLIPTUS PLUS 50/1000 MG 30 TAB,جليبتس بلس 50/1000 مجم 30 قرص,192.0


In [52]:
dataset_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83562 entries, 0 to 83561
Data columns (total 4 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   sku                          83562 non-null  int64  
 1   marketplace_product_name_ar  83562 non-null  object 
 2   seller_item_name             83562 non-null  object 
 3   price                        83562 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 2.6+ MB


## 1. Data Preprocessing

### a. Cleaning

In [76]:
def normalize_english(text):
    text = str(text).lower()
    # remove numbers (for master file only)
    text = re.sub(r'\d+', ' ', text)
    text = re.sub(r'[^a-z0-9\s]', ' ', text) # remove non-alphanumeric characters
    # text = re.sub(r'[^\w\s]', '', text) # remove punctuation
    text = re.sub(r'\s+', ' ', text) # remove multiple spaces
    text = text.strip()
    return text


def normalize_arabic(text):
    text = str(text).lower()
    text = re.sub(r"[\u064B-\u065F]", "", text)  # Remove Arabic diacritics (tashkeel)
    # text = re.sub(r'[^\u0621-\u064A\s]', '', text) # remove non-arabic characters
    text = re.sub(r'[^\u0621-\u064A0-9\s]', ' ', text) # remove non-arabic characters except for numbers
    text = re.sub(r'\s+', ' ', text) # remove multiple spaces

    text = re.sub( r'قرص|\bق\b|\bك\b|اقراص|كبسوله', 'كبسول', text)
    text = re.sub(r'([\u0600-\u06FF])\1+', r'\1', text) # remove arabic repetition
    text = re.sub(r'[إأآ]', 'ا', text)
    text = re.sub(r'ى', 'ي', text)
    text = re.sub(r'ة', 'ه ', text)
    text = re.sub(r'ؤ', 'و', text)
    text = re.sub(r'ئ', 'ي', text)

    # Separate numbers that stick to Arabic/English words
    text = re.sub(r"(\d+)([a-zA-Z\u0600-\u06FF]+)", r"\1 \2", text)  # Number followed by Arabic/English
    text = re.sub(r"([a-zA-Z\u0600-\u06FF]+)(\d+)", r"\1 \2", text)  # Arabic/English followed by number

    # Remove standalone Arabic/English characters (but not numbers)
    text = re.sub(r"\b[^\W\d]\b", "", text)
    
    text = re.sub(r'\b(?:سعر جديد|سعر|قديم|جديد|س جديد|س جدي|س ج|ركز)\b', '', text)# Remove specific unwanted phrases
    text = re.sub( r'مرهم|اكريم', 'كريم', text)
    text = re.sub(r'مليجرام|\bم\b|مجم', 'مجم', text)
    text = re.sub(r'جرام|جم', 'جم', text)
    text = re.sub( r'شرائط|شريطين', 'شريط', text)
    text = re.sub( r'امبولات|امبوله|حقن', 'امبول', text)
    text = re.sub( r'لبوس|لبوس اطفال', 'اقماع', text)
    text = re.sub(r"\s+", " ", text).strip() # remove multiple spaces
    return text

In [77]:
normalize_arabic("مرهم لعلاج2الجروح 10/200 جم")

'كريم لعلاج 2 الجروح 10 200 جم'

In [78]:
normalize_arabic("بيوفيت امبول جديددددد")

'بيوفيت امبول'

In [79]:
dataset_df['seller_item_name_clean'] = dataset_df['seller_item_name'].apply(normalize_arabic)

In [80]:
dataset_df.sample(10)

,sku,marketplace_product_name_ar,seller_item_name,price,seller_item_name_clean,seller_item_name_clean_spell
37533,666,ال-كارنيتين 20 كبسولة,ال كارنتين كبسول,72.0,ال كارنتين كبسول,ال كريم كبسول
55063,293,كارنيفيتا فورت 30 قرص,كارنيفيتا فورت 3شريط س/ج,177.0,كارنيفيتا فورت 3 شريط,كارنيفيتا فورت 3 شريط
5501,2132,كولوماك محلول موضعى 10 مل,كوللوماك محلول س جديد,18.0,كولوماك محلول,كولوماك محلول جي
69413,692,دولفين 12.5 مجم 10 أقماع,دولفن 12.5 لبوس شريطييين,36.0,دولفن 12 5 اقماع شريط,دي 12 5 اقماع شريط
61617,357,ديامونركتا 5 مجم 30 قرص,ديمنريكتا 5 محم 30اقراص,187.5,ديمنريكتا 5 محم 30 كبسول,دي 5 محم 30 كبسول
38004,977,الكابريس 10 مجم 30 قرص,الكابريس 10مجم 3شريط,78.0,الكابريس 10 مجم 3 شريط,الكابريس 10 مجم 3 شريط
18810,747,انتيكوكس 15مجم/3مل 6 امبول عضل,انتى كوكس 6امبوله /سعرجديد,78.0,انتي كوكس 6 امبول سعرجديد,ان كريم 6 امبول سي
4540,1323,موبيتيل 15 مجم 10 قرص,موبيتيل 15مجم 10قرص,23.5,موبيتيل 15 مجم 10 كبسول,موبيتيل 15 مجم 10 كبسول
16047,237,تلفاست 30مجم/5مل شراب معلق 100 مل,تلفاست شراب/س ج,50.0,تلفاست شراب,تلفاست شراب
62819,479,فيبراميسين 100 مجم 10 كبسولات,فيبراميسين 10 كبسول,42.5,فيبراميسين 10 كبسول,فيبراميسين 10 كبسول


In [95]:
def clean_master_file(text):
    text = str(text).lower()
    text = re.sub(r"[\u064B-\u065F]", "", text)  # Remove Arabic diacritics (tashkeel)

    text = re.sub(r'\d+', ' ', text) # remove numbers
    
    # text = re.sub(r'[^\u0621-\u064A\s]', '', text) # remove non-arabic characters
    text = re.sub(r'[^\u0621-\u064A0-9\s]', ' ', text) # remove non-arabic characters except for numbers
    text = re.sub(r'\s+', ' ', text) # remove multiple spaces

    text = re.sub( r'قرص|\bق\b|\bك\b|اقراص|كبسوله', 'كبسول', text)
    text = re.sub(r'([\u0600-\u06FF])\1+', r'\1', text) # remove arabic repetition
    text = re.sub(r'[إأآ]', 'ا', text)
    text = re.sub(r'ى', 'ي', text)
    text = re.sub(r'ة', 'ه ', text)
    text = re.sub(r'ؤ', 'و', text)
    text = re.sub(r'ئ', 'ي', text)

    # Separate numbers that stick to Arabic/English words
    text = re.sub(r"(\d+)([a-zA-Z\u0600-\u06FF]+)", r"\1 \2", text)  # Number followed by Arabic/English
    text = re.sub(r"([a-zA-Z\u0600-\u06FF]+)(\d+)", r"\1 \2", text)  # Arabic/English followed by number

    # Remove standalone Arabic/English characters (but not numbers)
    text = re.sub(r"\b[^\W\d]\b", "", text)
    
    text = re.sub(r'\b(?:سعر جديد|سعر|جديد|قديم|س جديد|س جدي|س ج|ركز)\b', '', text)# Remove specific unwanted phrases
    text = re.sub( r'مرهم|اكريم', 'كريم', text)
    text = re.sub( r'\bنقط\b', 'قطرة', text)
    text = re.sub(r'مليجرام|\bم\b|مجم', 'مجم', text)
    text = re.sub(r'جرام|جم', 'جم', text)
    text = re.sub( r'شرائط|شريطين', 'شريط', text)
    text = re.sub( r'امبولات|امبوله|حقن', 'امبول', text)
    text = re.sub( r'لبوس|لبوس اطفال', 'اقماع', text)
    text = re.sub(r"\s+", " ", text).strip() # remove multiple spaces
    return text

In [58]:
master_df['product_name_ar_clean'] = master_df['product_name_ar'].apply(clean_master_file)

In [59]:
master_df.sample(10)

,sku,product_name,product_name_ar,price,product_name_ar_clean
277,1394,ISOPTIN SR 240 MG 30 TAB,ايسوبتن ريتارد 240 مجم 30 قرص,97.50,ايسوبتن ريتارد مجم كبسول
987,438,BETADERM 0.1%CREAM 15 GM,بيتاديرم 0.1% كريم 15 جم,18.00,بيتاديرم كريم جم
58,121,COLOVERIN-D 30 TAB,كولوفيرين دى 30 قرص,105.00,كولوفيرين دي كبسول
962,3883,SWABIVENT NEBULIZER SOLUTION AMP. 20 * 2.5 ML,سوابيفنت 500 ميكرو محلول استنشاق 20 امبول,176.00,سوابيفنت ميكرو محلول استنشاق امبول
187,60,RIVAROSPIRE 10 MG 20 TAB,ريفاروسبير 10 مجم 20 قرص,254.00,ريفاروسبير مجم كبسول
899,1789,CIDOPHAGE 500 MG 50 STRIPES,سيدوفاج 500 مجم 50 شريط,550.00,سيدوفاج مجم شريط
393,2064,ATACAND 4 MG 14 TAB,اتاكاند 4 مجم 14 قرص,39.75,اتاكاند مجم كبسول
370,1863,STROKA 75 MG 30 TAB,ستروكا 75 مجم 30 قرص,145.00,ستروكا مجم كبسول
309,1517,GLYCODAL 30 CHEWABLE TAB,جليكودال 30 قرص مضغ,57.00,جليكودال كبسول مضغ
774,303,BETADINE ANTISEPTIC 10%SOLUTION 60 ML,بيتادين 10% محلول مطهر 60 مل موندى,55.00,بيتادين محلول مطهر مل موندي


In [ ]:
master_df['product_name_ar_clean'] = master_df['product_name_ar'].apply(normalize_arabic)
products_list = master_df['product_name_ar_clean'].str.split().str[0].apply(lambda x: x if len(x) >= 3 else None).dropna().tolist()
products_list = list(set(products_list))
print(len(products_list))
products_list

1000


0      [انافرونيل, 75, مجم, اس, ار, 20, كبسول]
1                    [لوبريكاف, شراب, 100, مل]
2                      [تومكس, بلس, 50, كبسول]
3              [تاروليمس, 0, 03, كريم, 15, جم]
4      [جليبتس, بلس, 50, 1000, مجم, 30, كبسول]
                        ...                   
995          [سولفاكس, بلس, مساج, جيل, 60, جم]
996              [مودابكس, 50, مجم, 30, كبسول]
997              [كابرون, 500, مجم, 20, كبسول]
998             [جارديانس, 10, مجم, 30, كبسول]
999              [ايموكس, 500, مجم, 16, كبسول]
Name: product_name_ar_clean, Length: 1000, dtype: object

In [97]:
# Create a list of all unique product names in the master file
master_product_names = master_df['product_name_ar_clean'].unique()
len(master_product_names)
# split the list into unique words
master_product_words = set(" ".join(master_product_names).split())
len(master_product_words)
master_product_words

{'مساج',
 'بانتازول',
 'اميلو',
 'اميجران',
 'ديبافالي',
 'جافيسكون',
 'ريباريل',
 'انتيكوكس',
 'دولفين',
 'بنادول',
 'جينت',
 'ريتارد',
 'ليفوهيستام',
 'ستروبانتين',
 '56',
 'روسيتور',
 'اتاكاند',
 'غسول',
 'توبراديكس',
 'ليبراكس',
 '550',
 'اقلام',
 'لعين',
 'ار',
 'بلس',
 'فتله',
 'توجيو',
 'بريدوكاين',
 'تيكانيز',
 'بريستافلام',
 'كولونا',
 'لي',
 '35',
 'بالحديد',
 'فيوسي',
 '40',
 'مفيس',
 'بكتيكلور',
 'اسيكلوفير',
 'بريجناكير',
 'ادفانسد',
 'كو',
 'فيلدافورمين',
 '81',
 'ريفو',
 'ملطف',
 'اوفيوسيديك',
 '32',
 '25',
 'اند',
 'استيل',
 'دايجيست',
 'رومانتيجرا',
 'يوريبان',
 '2800',
 'نستوجين',
 'سولوستار',
 'تادانيرفي',
 'ايفيبرونت',
 'تادالونج',
 'استرازنيكا',
 'ديابيتونورم',
 'ريفارسبير',
 'نازاكورت',
 'وجه',
 'سيبروفلوكساسين',
 'مطهر',
 'ليفانيك',
 'جوينت',
 'بانادول',
 'بلادوجرا',
 'فوليك',
 'كوليروز',
 'امريزول',
 'نافوبروكسين',
 'افازير',
 'انتريستو',
 'معباه',
 'نيرفيزام',
 'انسيلاكوكس',
 'جوينتا',
 'ملح',
 'كارنيتين',
 'ميلانوفري',
 'موديوريتك',
 '6',
 'انترافارما',
 'اوجم

In [105]:
product_names = master_df['product_name_ar_clean'].unique().tolist()
# Create a frequency dictionary (equal frequency for all names)
word_freq_dict = {name: 100 for name in products_list}
word_freq_dict

{'بانتازول': 100,
 'اميلو': 100,
 'اميجران': 100,
 'ديبافالي': 100,
 'جافيسكون': 100,
 'ريباريل': 100,
 'انتيكوكس': 100,
 'دولفين': 100,
 'بنادول': 100,
 'ليفوهيستام': 100,
 'ستروبانتين': 100,
 'روسيتور': 100,
 'اتاكاند': 100,
 'توبراديكس': 100,
 'ليبراكس': 100,
 'توجيو': 100,
 'بريدوكاين': 100,
 'تيكانيز': 100,
 'بريستافلام': 100,
 'كولونا': 100,
 'فيوسي': 100,
 'بكتيكلور': 100,
 'بريجناكير': 100,
 'اسيكلوفير': 100,
 'فيلدافورمين': 100,
 'ريفو': 100,
 'اوفيوسيديك': 100,
 'استيل': 100,
 'دايجيست': 100,
 'رومانتيجرا': 100,
 'يوريبان': 100,
 'نستوجين': 100,
 'تادانيرفي': 100,
 'ايفيبرونت': 100,
 'تادالونج': 100,
 'ديابيتونورم': 100,
 'ريفارسبير': 100,
 'نازاكورت': 100,
 'سيبروفلوكساسين': 100,
 'ليفانيك': 100,
 'جوينت': 100,
 'بانادول': 100,
 'بلادوجرا': 100,
 'فوليك': 100,
 'كوليروز': 100,
 'امريزول': 100,
 'نافوبروكسين': 100,
 'افازير': 100,
 'انتريستو': 100,
 'نيرفيزام': 100,
 'انسيلاكوكس': 100,
 'جوينتا': 100,
 'ميلانوفري': 100,
 'موديوريتك': 100,
 'اوجم': 100,
 'كريستوليب': 100,
 'ال

In [ ]:
from Levenshtein import distance as levenshtein_distance

def suggest_corrections(misspelled_word, top_n=1, max_distance=3):
    suggestions = []
    for word in word_freq_dict:
        # Calculate edit distance between the misspelled word and the product name
        edit_dist = levenshtein_distance(misspelled_word, word)
        if edit_dist <= max_distance:
            suggestions.append((word, word_freq_dict[word], edit_dist))
    # Sort suggestions by edit distance and frequency
    suggestions.sort(key=lambda x: (x[2], -x[1]))
    return suggestions[0][0] if suggestions else misspelled_word
    
# Example usage
misspelled_word = "فلكسوفان"
corrections = suggest_corrections(misspelled_word, word_freq_dict)
print(f"Suggestions for '{misspelled_word}': {corrections}")

Suggestions for 'فلكسوفان': فليكسوفان


In [107]:
from wordfreq import word_frequency, top_n_list
from collections import defaultdict

# Get the top N most frequent Arabic words
top_arabic_words = top_n_list('ar', 100000)  # Adjust the number as needed

# Create a frequency dictionary
word_freq_dict = {word: word_frequency(word, 'ar') for word in master_product_words}

# Function to suggest corrections for a misspelled word
def suggest_corrections(misspelled_word, word_freq_dict, top_n=5):
    suggestions = []
    for word in word_freq_dict:
        if word.startswith(misspelled_word[0]):  # Simple heuristic: same starting letter
            suggestions.append((word, word_freq_dict[word]))
    # Sort suggestions by frequency (higher frequency first)
    suggestions.sort(key=lambda x: x[1], reverse=True)
    return [word for word, freq in suggestions[:top_n]]

# Example usage
misspelled_word = "فلكسوفان"
corrections = suggest_corrections(misspelled_word, word_freq_dict)
print(f"Suggestions for '{misspelled_word}': {corrections}")

Suggestions for 'فلكسوفان': ['فيلم', 'فيل', 'فلو', 'فيتامين', 'فورت']


In [108]:
def fix_misspelled_words_in_string(input_string, max_distance=3):
    """
    Fix misspelled words in a string while leaving correctly spelled words unchanged.
    """
    # Tokenize the input string into words
    words = input_string.split()
    
    # Correct each word if it is misspelled
    corrected_words = []
    for word in words:
        # Check if the word is in the frequency dictionary (assume it's correct if it is)
        if word in word_freq_dict:
            corrected_words.append(word)
        else:
            # If the word is not in the dictionary, suggest a correction
            corrected_word = suggest_corrections(word, word_freq_dict, max_distance)
            # If suggestions are returned, take the first one
            if corrected_word:
                corrected_words.append(corrected_word[0])
            else:
                corrected_words.append(word)
    
    # Reconstruct the string
    corrected_string = ' '.join(corrected_words)
    return corrected_string

dataset_df['seller_item_name_clean_spell'] = dataset_df['seller_item_name_clean'].apply(lambda x: fix_misspelled_words_in_string(str(x)))

In [109]:
fix_misspelled_words_in_string("اوندالينزا 4 مجم كبسول")

'ان 4 مجم كبسول'

In [110]:
dataset_df.sample(10)

,sku,marketplace_product_name_ar,seller_item_name,price,seller_item_name_clean,seller_item_name_clean_spell
24361,4403,دوزين 1 مجم 20 قرص,دوزين 1مجم اقراص سعر 14 جنيه,14.0,دوزين 1 مجم كبسول 14 جنيه,دوزين 1 مجم كبسول 14 جي
71389,1631,افيميو قطرة عين 10 مل,ايفيميو قطرة للعين,44.0,ايفيميو قطره لعين,ان قطره لعين
53921,2082,بينوكس 0.4% قطرة عين 10 مل,بينوكس نقط,23.0,بينوكس نقط,بينوكس نقط
35446,2696,اتورستات 40 مجم 14 قرص,اتورستات 40 م 14 قرص د,76.0,اتورستات 40 14 كبسول,اتورستات 40 14 كبسول
62596,478,سيريتايد ديسكس 250/50 مكجم 60 جرعة,سيريتيد 250 ديسكس 60جرعة**,257.0,سيريتيد 250 ديسكس 60 جرعه,سي 250 ديسكس 60 جرعه
69945,84,اوميز 20 مجم 14 كبسول,اوميز 20مجم 14كبسول سعر جديد,56.0,اوميز 20 مجم 14 كبسول,اوميز 20 مجم 14 كبسول
35975,3586,ميليتوفكس مت 5/1000 مجم 30 قرص,MELLITOFIX MET 5MG/ 1000 MG 30 TAB,163.5,5 1000 30,5 1000 30
69580,744,كيتولاك 10 مجم 20 قرص,كيتولاك 10مج20ق س ج/العامرية,38.0,كيتولاك 10 مج 20 العامريه,كيتولاك 10 مج 20 العامريه
63424,53,زيثرون 500 مجم 5 قرص,زيثرون 500 -5قرص س ج,86.0,زيثرون 500 5 كبسول,زيثرون 500 5 كبسول
40689,1863,ستروكا 75 مجم 30 قرص,ستروكا اقراص/ابكس,145.0,ستروكا كبسول ابكس,ستروكا كبسول ان


In [119]:
def extract_and_filter_words(text):
    # Use regex to find all words (excluding numbers)
    words = re.findall(r'\b[^\d\W]+\b', text)
    # Filter out words with fewer than 3 letters
    return [word for word in words if len(word) >= 3]

# Apply the function to the 'product_name_ar' column
all_words = master_df['product_name_ar'].apply(extract_and_filter_words).explode().dropna().tolist()

In [121]:
print(len(all_words))
all_words

3099


['انافرونيل',
 'مجم',
 'قرص',
 'لوبريكاف',
 'شراب',
 'تومكس',
 'بلس',
 'قرص',
 'تاروليمس',
 'مرهم',
 'جليبتس',
 'بلس',
 'مجم',
 'قرص',
 'دالاسين',
 'مجم',
 'كبسول',
 'باي',
 'بروفينيد',
 'مجم',
 'قرص',
 'باي',
 'كيتوجيسيك',
 'مجم',
 'قرص',
 'بروتوفيكس',
 'مجم',
 'قرص',
 'ارثروفاست',
 'مجم',
 'قرص',
 'ارث',
 'فرى',
 'مجم',
 'قرص',
 'انافرونيل',
 'مجم',
 'قرص',
 'فليكتور',
 'مجم',
 'كبسول',
 'ديباكين',
 'مجم',
 'محلول',
 'بالفم',
 'بانتازول',
 'مجم',
 'قرص',
 'اتاكاند',
 'مجم',
 'قرص',
 'هيموجيت',
 'مجم',
 'شراب',
 'ليليبيل',
 'مجم',
 'قرص',
 'موديوريتك',
 'مجم',
 'قرص',
 'اميلو',
 'مجم',
 'قرص',
 'اتورستات',
 'مجم',
 'قرص',
 'كيريلا',
 'مرهم',
 'كروميوم',
 'ميباكو',
 'كبسول',
 'البندازول',
 'مجم',
 'قرص',
 'نافوبروكسين',
 'مجم',
 'اقماع',
 'توبمود',
 'مجم',
 'كبسولة',
 'ديكلاك',
 'مجم',
 'قرص',
 'التراكايين',
 'جيل',
 'كولوفيرين',
 'قرص',
 'فيروجلوبين',
 'شراب',
 'ميلجا',
 'قرص',
 'بلافيكس',
 'مجم',
 'قرص',
 'بانادول',
 'اكسترا',
 'قرص',
 'فيروجلوبين',
 'كبسول',
 'كونفنتين',
 'مجم',
 'ك

In [122]:
from fuzzywuzzy import process

def correct_spelling(input_string):
    words = input_string.split()
    corrected_words = []
    for word in words:
        match, score = process.extractOne(word, all_words)
        if score >= 80:
            corrected_words.append(match)
        else:
            corrected_words.append(word)

    corrected_string = ' '.join(corrected_words)
    return corrected_string    

In [123]:
correct_spelling("اوندالينزا 4 مجم كبسول")

'اوندالينز 4 مجم كبسول'

In [124]:
dataset_df['seller_item_name_clean_spell'] = dataset_df['seller_item_name_clean'].apply(correct_spelling)

KeyboardInterrupt: 